# 8.2.1 Quick Demo: Creating a RAG Application with Amazon Bedrock Knowledge Base

<div style="background-color: #f8f9fa; border: 1px solid #e9ecef; border-radius: 8px; padding: 10px; margin: 10px;">
<strong>📋 Workshop Contents</strong>
<ul style="line-height: 1.2;">
<li><a href="#What-Youll-Learn">What You'll Learn</a></li>
<li><a href="#Prerequisites">Prerequisites</a></li>
<li><a href="#Step-1-Access-the-Amazon-Bedrock-Console">Step 1: Access the Amazon Bedrock Console</a></li>
<li><a href="#Step-2-Create-a-Knowledge-Base">Step 2: Create a Knowledge Base</a></li>
<li><a href="#Step-3-Optionally-Add-Additionally-Data-Sources">Step 3: Optionally Add Additionally Data Sources</a></li>
<li><a href="#Step-4-Monitor-Ingestion-Process">Step 4: Monitor Ingestion Process</a></li>
<li><a href="#Step-5-Test-Your-Knowledge-Base">Step 5: Test Your Knowledge Base</a></li>
<li><a href="#Step-6-Integrate-with-Your-Applications">Step 6: Integrate with Your Applications</a></li>
<li><a href="#Benefits-of-Using-Bedrock-Knowledge-Base">Benefits of Using Bedrock Knowledge Base</a></li>
<li><a href="#Next-Steps">Next Steps</a></li>
</ul>
</div>

Welcome to the quickest way to build a RAG application! In this notebook, we'll explore how to create a knowledge base using the Amazon Bedrock console, which provides a no-code approach to implementing RAG.

## What You'll Learn
- How to create a knowledge base in Amazon Bedrock
- How to upload and process documents
- How to test queries against your knowledge base
- How to integrate the knowledge base with your applications

## Prerequisites
- AWS account with Amazon Bedrock access
- Sample documents to upload (PDFs, Word docs, text files)
- Basic understanding of RAG concepts (covered in [section 8.1](../8.1_Data_as_Your_Strategic_Asset_for_GenAI_Applications/README.md))
- Jupyter Notebook: You can launch a [free tier Amazon SageMaker Jupyter Notebook](../../1_Getting_Started_with_AWS/1.4_Setting_up_Your_Cookbook_Environment/README.MD)

## Step 1: Access the Amazon Bedrock Console

1. Sign in to the [AWS Management Console](https://us-east-1.console.aws.amazon.com/console/home?region=us-east-1)
2. Navigate to [Amazon Bedrock](https://us-east-1.console.aws.amazon.com/bedrock/)
3. Ensure you have model access to at least one foundation model (e.g., Claude, Titan)

![Bedrock Console](images/8.2-1-model-access.gif)
*Amazon Bedrock console showing the Knowledge bases section*

## Step 2: Create a Knowledge Base

1. In the Bedrock console, navigate to "[Knowledge bases](https://us-east-1.console.aws.amazon.com/bedrock/home?region=us-east-1#/knowledge-bases)" in the left navigation
2. Click "Create knowledge base"
3. Provide a name and optional description for your knowledge base
4. Provide IAM permissions. Choose: Create and use a new service role
5. Choose data source type (e.g. Amazon S3)
6. For S3, specify the bucket and prefix
7. Configure chunking settings (or use defaults)
8. Choose a foundation model for embeddings (e.g., Amazon Titan Embeddings)
9. Configure vector store settings. For testing purpose, recommend choose "Quick create a new vector store". Then, choose Amazon Aurora PostgreSQL Serverless as the vector store.
10. Review and create your knowledge base

![Creating a Knowledge Base](images/8.2-2-create-bedrock-knowledge-base.gif)
*Step-by-step process of creating a knowledge base*

## Step 3: Optionally Add Additionally Data Sources

1. Once your knowledge base is created, click "Add data source"
2. Choose your data source type, e.g. S3 bucket (for bulk document upload), Direct upload (for individual files)
3. For S3, specify the bucket and prefix
4. For direct upload, select files from your computer
5. Configure chunking settings (or use defaults)
6. Start the ingestion process

```python
# If you prefer to upload documents programmatically to S3 first:
import boto3
import os

# Create S3 client
s3 = boto3.client('s3')

# Define bucket and document directory
bucket_name = 'your-bucket-name'
document_dir = 'path/to/your/documents'

# Upload all documents in the directory
for filename in os.listdir(document_dir):
    if filename.endswith(('.pdf', '.docx', '.txt')):
        file_path = os.path.join(document_dir, filename)
        s3_key = f'knowledge-base-docs/{filename}'
        
        print(f"Uploading {filename} to S3...")
        s3.upload_file(file_path, bucket_name, s3_key)
        
print("All documents uploaded successfully!")
```

## Step 4: Monitor Ingestion Process

1. Click "Sync" associated with the data source
2. After starting the ingestion, you can monitor its progress in the console
3. The process includes:
   - Document parsing
   - Text extraction
   - Chunking
   - Embedding generation
   - Vector storage
4. Wait for the status to change to "Available" before proceeding

![Ingestion Process](images/8.2-3-sync-data-source.gif)
*Monitoring the document ingestion process*

## Step 5: Test Your Knowledge Base

1. Once ingestion is complete, click on the "Test" tab
2. Enter a query related to your documents
3. Select a foundation model for response generation (e.g., Claude, Titan)
4. Click "Run" to see the results
5. Review both the retrieved chunks and the generated response

![Testing Knowledge Base](images/8.2-4-test-bedrock-knowledge-base.gif)
*Testing the knowledge base with a sample query*

## Step 6: Integrate with Your Applications

Now that your knowledge base is working, you can integrate it with your applications using the AWS SDK:

Query the knowledge base programmatically using the AWS SDK to retrieve relevant document chunks and their metadata. The function returns similarity scores and source locations for each retrieved chunk, enabling you to understand the relevance and origin of the information with confidence scores and source document references.

In [ ]:
import boto3
import json

# Initialize Bedrock client
bedrock_agent_runtime = boto3.client(
    service_name='bedrock-agent-runtime',
    region_name='us-east-1'  # Change to your region
)

# Knowledge base details
knowledge_base_id = 'your-knowledge-base-id'  # Get this from the console

# Function to query the knowledge base
def query_knowledge_base(query_text):
    response = bedrock_agent_runtime.retrieve(
        knowledgeBaseId=knowledge_base_id,
        retrievalQuery={
            'text': query_text
        },
        retrievalConfiguration={
            'vectorSearchConfiguration': {
                'numberOfResults': 3  # Number of chunks to retrieve
            }
        }
    )
    
    return response['retrievalResults']

# Test the function
results = query_knowledge_base("What are the key features of Amazon Aurora?")

# Print the retrieved chunks
for i, result in enumerate(results):
    print(f"Result {i+1}:")
    print(f"Content: {result['content']['text']}")
    print(f"Source: {result['location']['s3Location']['uri']}")
    print(f"Score: {result['score']}")
    print("-" * 50)

Complete the RAG pipeline by combining document retrieval with response generation using foundation models. The function retrieves relevant chunks, constructs a context-aware prompt, and generates a comprehensive answer using Claude or other Bedrock models with the complete query-response cycle using contextual information from your knowledge base.

In [ ]:
# Generate a response using the retrieved information
def generate_rag_response(query_text):
    # Step 1: Retrieve relevant chunks
    retrieval_results = query_knowledge_base(query_text)
    
    # Step 2: Extract text from retrieved chunks
    retrieved_content = [result['content']['text'] for result in retrieval_results]
    context = "\n\n".join(retrieved_content)
    
    # Step 3: Generate response using Bedrock model
    bedrock_runtime = boto3.client(
        service_name='bedrock-runtime',
        region_name='us-east-1'  # Change to your region
    )
    
    prompt = f"""
    Human: Answer the following question based on the provided context. 
    If you cannot answer the question based on the context, say "I don't have enough information to answer this question."
    
    Context:
    {context}
    
    Question: {query_text}
    
    Assistant:
    """
    
    response = bedrock_runtime.invoke_model(
        modelId='anthropic.claude-v2',  # Or your preferred model
        contentType='application/json',
        accept='application/json',
        body=json.dumps({
            "prompt": prompt,
            "max_tokens_to_sample": 500,
            "temperature": 0.7,
            "top_p": 0.9,
        })
    )
    
    response_body = json.loads(response['body'].read())
    return response_body['completion']

# Test the complete RAG pipeline
query = "What are the key benefits of using Aurora PostgreSQL?"
response = generate_rag_response(query)
print(f"Query: {query}\n")
print(f"Response: {response}")

## Benefits of Using Bedrock Knowledge Base

- **No infrastructure to manage**: Amazon manages the vector database and scaling
- **Quick setup**: Create a working RAG application in minutes
- **Automatic document processing**: Handles parsing, chunking, and embedding generation
- **Seamless integration**: Easy to connect with other AWS services
- **Cost-effective**: Pay only for what you use

## Limitations

- Less customization compared to building your own solution
- Limited control over chunking strategies
- Fixed embedding models
- May not be suitable for specialized domains requiring custom processing

## Next Steps

Now that you've created a basic RAG application using Amazon Bedrock Knowledge Base, you can:

1. Explore more advanced features in the [Bedrock documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html)
2. Learn how to build a custom RAG solution with Aurora PostgreSQL in [8.2.2_aurora_pgvector_rag_application.ipynb](8.2.2_aurora_pgvector_rag_application.ipynb)
3. Apply these concepts to build specialized applications like a [Claim Processing Agent](../8.3_Claim_Processing_Agent/README.MD)

## Additional Resources 📚

### Amazon Bedrock & RAG
- [Amazon Bedrock Knowledge Base User Guide](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html)
- [Amazon Bedrock API Reference](https://docs.aws.amazon.com/bedrock/latest/APIReference/)
- [RAG with Amazon Bedrock Workshop](https://catalog.workshops.aws/building-with-amazon-bedrock/)

### Foundation Models & Embeddings
- [Amazon Titan Embeddings](https://docs.aws.amazon.com/bedrock/latest/userguide/titan-embedding-models.html)
- [Anthropic Claude Models](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude.html)
- [Model Access and Pricing](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)

### Vector Databases & Search
- [Amazon OpenSearch Serverless](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/serverless.html)
- [Vector Search Best Practices](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/knn.html)
- [Document Processing and Chunking](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-ds.html)